# Accelerating token generation of the Hugging Face Gemma Model with Transformer Engine

Generative AI has made remarkable strides in recent years, with Large Language Models (LLMs) like ChatGPT at the forefront. These models have revolutionized how we interact with machine-generated content, providing capabilities that range from writing assistance to complex decision support. The core functionality of these models is the generation process, which involves predicting the next token in a sequence based on the preceding text. This task is critical for applications such as automated content creation, translation, and more, emphasizing the importance of efficient implementation.


<figure align="center">
<img src="./media/generation_animation.gif" alt="" >
<figcaption>
Animation 1: Hugging Face Gemma model token generation.
</figcaption>
</figure>

For those seeking a deeper understanding of text generation mechanisms in Transformers, it is recommended to check out the [HuggingFace generation tutorial](https://huggingface.co/docs/transformers/llm_tutorial).

In the previous tutorials on [Llama](../te_llama/tutorial_accelerate_hf_llama_finetuning_with_te.ipynb) and [Gemma](./tutorial_accelerate_hf_gemma_finetuning_with_te.ipynb), it was demonstrated how finetuning can be accelerated using the Transformer Engine's `TransformerLayer`. Building on this foundation, the current objective is to enhance the generation speed of the Gemma model.

This tutorial will introduce and explain several advanced features of the Transformer Engine that contribute to this goal:

#### 1. From vanilla KV-caching to Paged Attention for inference in Transformer Engine

The original [Attention mechanism](https://arxiv.org/pdf/1706.03762) ushered an era of Large Language Models but the same attention mechanism if used for deployment in inference scenarios can be computationally wasteful. It is primarily due to a lot of redundant computation that happens in attention when the Transformer models are used autoregressively to compute the next token. Several tutorials on the internet explain in detail how KV Caching helps to redunce that redundant computation e.g. [tutorial 1](https://magazine.sebastianraschka.com/p/coding-the-kv-cache-in-llms), [tutorial 2](https://medium.com/@joaolages/kv-caching-explained-276520203249), etc.


Further, even though the performance benefit of KV Cache is immense, it comes at the cost of increased memory usage which becomes a problem especially for longer context lengths. The major problems are: 
1. Internal fragmentation
2. External Fragmentation

[Paged Attention](https://arxiv.org/pdf/2309.06180) solves this by treating the KV cache as a virtual memory with the actual physical blocks being much smaller than the overall cache size which makes it easier to swap them in and out of GPU HBM as needed - very similar to how Operating Systems implement virtual memory to swap the individual pages in and out of the CPU RAM.


Transformer Engine allows users to use both the forms of KV Caching and the results in this tutorial are posted for both those cases.
1. Non-paged
2. Paged


#### **2. CUDA Graphs API.**

The speed of GPUs is increasing at a rapid pace. It turns out that sometimes the runtime of kernels is shorter than the time it takes for the CPU to submit them, which can lead to significant overhead. CUDA Graphs can address this issue. When certain kernels are executed repeatedly, it allows us to record and replay them with less CPU involvement. This becomes particularly useful in applications like token generation, where a `TransformerLayer` is run for every token that needs to be generated.

One can read more about CUDA Graphs [here](https://developer.nvidia.com/blog/cuda-graphs/).

PyTorch exposes graphs via a raw `torch.cuda.CUDAGraph` class and two convenience wrappers: `torch.cuda.graph` and `torch.cuda.make_graphed_callables`. More information about the cuda graphs in Pytorch can be found [here](https://pytorch.org/blog/accelerating-pytorch-with-cuda-graphs/).

<figure align="center">
<img src="./media/graphs.svg" alt="">
<figcaption>
Figure 2: CUDA Graphs reduce the overhead generated by the long time it takes to launch a single kernel. It enables the recording and replaying of subsequent launches, thus reducing the total time used by the CPU.
</figcaption>
</figure>


#### **3. FP8 Weights Calibration.**

Assuming that the model is trained in FP32/BF16 precision and the goal is to execute it in FP8 precision, the process isn't straightforward due to the absence of appropriate FP8 scaling factors. In this scenario, FP8 calibration becomes essential. By conducting several forward passes on sample data, the FP8 scaling parameters can be computed. This calibration allows the model to operate correctly in FP8 precision.

It is highly recommended to familiarize oneself with the [tutorial](../../examples/fp8_primer.ipynb) on FP8 precision to understand the importance of proper scaling factors.


<figure align="center">
<img src="./media/calibration.svg" alt="">
<figcaption>
Figure 3:
If the model is trained in BF16/FP32, it does not include the computed FP8 scaling factors. When it is run under <b>fp8_autocast()</b>, the value of these scaling factors will default to their initial values, which can cause numerical errors. Weight calibration involves calculating FP8 scaling factors from higher precision forward passes. Once these factors are computed, the model becomes numerically stable. 
</figcaption>
</figure>

#### **4. FP8 Model Weights.**

The typical approach is to store weights in higher precision and then cast them to fp8 before operations. This may prevent accuraccy drops in training. However, for inference, this level of precision is not necessary.

The TransformerEngine includes a wrapper `fp8_model_​init`, which allows for the creation of models that store only the FP8 copy of the weights. This eliminates the need to cast from higher precision to BF16, saving time in this casting process. 

<figure align="center">
<img src="./media/fp8_model_init.svg" alt="">
<figcaption>
Figure 4: Model under <b>fp8_autocast()</b> stores weights in high precision by default, and casts them if needed. It can leads to slowdown and increased memory usage. Using <b>fp8_model_init()</b> results in storing weight in FP8.
</figcaption>
</figure>

#### Benchmarking

We'll evaluate the generation time across one benchmark: generation with context phase max sequence length = 128, batch size = 64 and number of generated tokens = 896 on random texts with random lengths.

<div class="alert alert-info">
<b>Note</b>
    
This tutorial focuses on showcasing the mentioned features of Transformer Engine in the context of token generation. It's important to note, however, that NVIDIA provides [TensorRT](https://developer.nvidia.com/tensorrt), which is optimized for inference tasks and should be considered for such use cases.
</div>

## Dependencies for this tutorial

Following files and media are necessary to effectively run this tutorial:

1. `te_gemma.py`
    - This file contains the code to load a Hugging Face Gemma checkpoint weights in Transformer Engine's `TransformerLayer` instead of Hugging Face's `GemmaDecoderLayer`. Further, it contains necessary abstractions like a subclass of `GemmaForCausalLM` - `TEGemmaForCausalLM` that is used for generation with Transformer Engine's `TransformerLayer`, CUDA Graphs and weight calibration for generation in FP8 precision.
2. `te_gemma_loading_weights.py`
    - This file contains logic of mapping the parameters from `GemmaDecoderLayer` into the `TransformerLayer`.
3. `utils.py`
    - This file contains the code related to dataloading, hyperparameters, setting up model/optimizers/accelerator, model training and other miscellaneous tasks like restarting the jupyter notebook from within the cell. 
4. `requirements.txt`
    - This file contains necessary Python packages for this tutorial.
5. `media/`
    - This directory contains the images and other artefacts used in the following tutorial.

In [1]:
%pip install -r requirements.txt

import torch

cudnn_version = torch.backends.cudnn.version()
assert cudnn_version >= 90100, "cuDNN version >= 9.1.0 is needed to run this tutorial."

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 113.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 150.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 159.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 156.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 79.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
  Attempting uninstall: dill
    Found existing installation: dill 0.4.0━━━━━━━━━━━━━━━━━━━━━━━  5/12 [dill]
    Uninstalling dill-0.4.0:m╸━━━━━━━━━━━━━━━━━━━━━━━  5/12 [dill]
      Successfully uninstalled dill-0.4.0━

## [Baseline] Running Hugging Face generation with Gemma model

HuggingFace Transformers library offers generation API. 
HuggingFace generation for the Gemma model will be used as a baseline.

In [1]:
# Restart the notebook (to flush the GPU memory)
from utils import restart_jupyter_notebook

restart_jupyter_notebook()

from utils import *

# Default hyperparams, also defined in `utils.py` in class `Hyperparameters`
# !!! `model_name` attr must point to the location of the model weights !!!
# Weights can be downloaded from: https://huggingface.co/google/gemma-7b.
# Weights should be in the *.safetensors HF format, not in the original format.
hyperparams.model_name = "/ckpt/gemma-7b-hf/"  # <== Add model weight location here e.g. "/path/to/downloaded/gemma/weights"
hyperparams.batch_size = 64

model = init_baseline_model(hyperparams)

print_sample_of_generated_texts(model, hyperparams)
benchmark_generation(model, hyperparams)
print(f"Max memory allocated: {torch.cuda.max_memory_allocated()/1024**2} MB")

/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.45s/it]


============================== Generation example 1 ==============================
Prompt:
Here are the two facts about GPUs:
Generated text:


1. GPUs are very good at doing the same thing over and over again.
2. GPUs are very bad at doing different things at the same time.

The first fact is why GPUs are so good at graphics. The second fact is
============================== Generation example 2 ==============================
Prompt:
Some facts about NVIDIA:
Generated text:


* NVIDIA is a global technology company that designs and develops high-performance computer graphics and video processing chips.
* The company was founded in 1993 and is headquartered in Santa Clara, California.
* NVIDIA's products are
Benchmarking for batch_size = 64 and prefill tokens = 20 and max new tokens = 492
Time: 45.11 s.
Max memory allocated: 73534.830078125 MB


Let's put this time into the table for later comparison.

| Models                                                      | Time (s) | Speedup |  
|-------------------------------------------------------------|---------------------------------------|--------------------------------------|
| HF (baseline)                                               | 45.11      | 1                         |

## [Improvement 1] Using TransformerLayer from Transformer Engine instead of GemmaDecoderLayer.

As in the [Gemma](./tutorial_accelerate_hf_gemma_finetuning_with_te.ipynb) finetuning tutorial, a GemmaDecoderLayer is substituted by a tuned TransformerLayer from the Transformer Engine. Let's run it and compare the time with the baseline.

In [2]:
# Restart the notebook (to flush the GPU memory)
from utils import restart_jupyter_notebook

restart_jupyter_notebook()

from utils import *

hyperparams.model_name = "/ckpt/gemma-7b-hf/"  # <== Add model weight location here e.g. "/path/to/downloaded/gemma/weights"
hyperparams.batch_size = 64
hyperparams.is_paged = False
hyperparams.max_seq_length = 512

model = init_te_gemma_model(hyperparams)

print_sample_of_generated_texts(model, hyperparams)
benchmark_generation(model, hyperparams)

/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


batch_size: 64,                     max_input_sequence_len: 512,                     lengths: tensor([ 9,  6, 10,  6,  9,  6, 10,  6,  9,  6, 10,  6,  9,  6, 10,  6,  9,  6,
        10,  6,  9,  6, 10,  6,  9,  6, 10,  6,  9,  6, 10,  6,  9,  6, 10,  6,
         9,  6, 10,  6,  9,  6, 10,  6,  9,  6, 10,  6,  9,  6, 10,  6,  9,  6,
        10,  6,  9,  6, 10,  6,  9,  6, 10,  6], device='cuda:0')
============================== Generation example 1 ==============================
Prompt:
Here are the two facts about GPUs:
Generated text:


1. They are very good at doing a lot of the same thing at the same time.
2. They are very bad at doing different things at the same time.

The first fact is why they are so good at graphics. The second
============================== Generation example 2 ==============================
Prompt:
Some facts about NVIDIA:
Generated text:


* NVIDIA is a global technology company that designs and builds the world’s most advanced computer chips and systems for

With just using Transformer Engine with default cuDNN attention, a speedup of **3.7x** was obtained. Neat!

| Models                                                      | Time (s) | Speedup (non-paged)
|-------------------------------------------------------------|---------------------------------------|--------------------------------------|
| HF (baseline)                                               | 45.11      | 1                         |
| TE (subsitution of GemmaDecoderLayer with te.TransformerLayer)                                              | 12.10      | 3.7                         | 

## [Improvement 2] Speeding up generation with CUDA Graphs

TransformerEngine includes a function `transformer_engine.pytorch.make_graphed_callables`, which functions similarly to the corresponding feature in PyTorch. It is capable of recording any modules from the Transformer Engine. Below is a code excerpt from `te_gemma.py` from class `TEGemmaForCausalLMCudaGraphs`:
```
    def __init__(self, config : GemmaConfig):
            (...)
            
            # Here "the trick" happens. We override methods from TEGemmaForCausalLM
            # with their recorded version. After invocation of each of them,
            # captured graph will be replayed with minimal usage of CPU,
            # what will lead to huge speedup.
            (...)
            self._model_context_phase = 
                self.record_graph(self._model_context_phase, self.hidden_states_buffer) # CUDA Graphs recording

            (...)        
            self._model_generation_phase = 
                self.record_graph(self._model_generation_phase, self.generation_buffer) # CUDA Graphs recording

    @torch.no_grad()
    def record_graph(self, function, input_tensor):
        (...)
        # function is invoked on argument (self.hidden_states,) and all kernels are recorded.
        # record_graph() returns captured function, which can be run later with minimal use of th CPU.
        fp8_format = Format.HYBRID
        fp8_recipe = DelayedScaling(fp8_format=fp8_format, amax_history_len=32, amax_compute_algo="max")
        with autocast(dtype=torch.bfloat16, cache_enabled=False):
            graphed_function = te.pytorch.make_graphed_callables(
                function, 
                (input_tensor,), 
                fp8_enabled=True, 
                fp8_recipe=fp8_recipe, 
                allow_unused_input=True,
                num_warmup_iters=3
            )
        return graphed_function
```

It is strongly reccomended to review the entire code of the class `TEGemmaForCausalLMCudaGraphs`. Let's now proceed to evaluate the performance improvement offered by CUDA Graphs.

In [1]:
# Restart the notebook (to flush the GPU memory)
from utils import restart_jupyter_notebook

restart_jupyter_notebook()


from utils import *

hyperparams.model_name = "/ckpt/gemma-7b-hf/"  # <== Add model weight location here e.g. "/path/to/downloaded/gemma/weights"
hyperparams.generation_cuda_graphs = True
hyperparams.max_seq_length = 512
hyperparams.batch_size = 64

# It is necessary to preallocate a static buffer.
# CUDA graphs require static input tensors for every kernel.
# This approach may result in a slight increase in memory consumption;
# however, the substantial speedup achieved makes it worthwhile.
hyperparams.cuda_graphs_static_batch_size = 64
hyperparams.cuda_graphs_static_max_seq_len = 512
hyperparams.cuda_graphs_static_max_context_len = 512
model = init_te_gemma_model(hyperparams)

print_sample_of_generated_texts(model, hyperparams)
benchmark_generation(model, hyperparams)

/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


batch_size: 64,                     max_input_sequence_len: 512,                     lengths: tensor([ 9,  6, 10,  6,  9,  6, 10,  6,  9,  6, 10,  6,  9,  6, 10,  6,  9,  6,
        10,  6,  9,  6, 10,  6,  9,  6, 10,  6,  9,  6, 10,  6,  9,  6, 10,  6,
         9,  6, 10,  6,  9,  6, 10,  6,  9,  6, 10,  6,  9,  6, 10,  6,  9,  6,
        10,  6,  9,  6, 10,  6,  9,  6, 10,  6], device='cuda:0')
============================== Generation example 1 ==============================
Prompt:
Here are the two facts about GPUs:
Generated text:


1. They are very good at doing a lot of the same thing at the same time.
2. They are very bad at doing different things at the same time.

The first fact is why they are so good at graphics. The second
============================== Generation example 2 ==============================
Prompt:
Some facts about NVIDIA:
Generated text:


* NVIDIA is a global technology company that designs and builds the world’s most advanced computer chips and systems for

A speed up of **7x** was obtained by using CUDA Graphs with TE Transformer Layer.

| Models                                                      | Time (s) | Speedup |  
|-------------------------------------------------------------|---------------------------------------|--------------------------------------|
| HF (baseline)                                               | 45.11      | 1                         |
| TE (subsitution of GemmaDecoderLayer with te.TransformerLayer)                                              | 12.10      | 3.7                         | 
| TE (te.TransformerLayer) + CUDA Graphs                                             | 6.41      | 7                         |  


Let's look at the screenshots from *NVIDIA Nsight System* profiler to see where this speedup comes from:

<figure align="center">
<img src="./media/graphs_1.png" width="80%">
<figcaption>
Figure 5: Without CUDA Graphs. One can see that GPU (blue) is idle for big portion of the time.
</figcaption>
</figure>

<figure align="center">
<img src="./media/graphs_2.png" width="80%">
<figcaption>
Figure 6: With CUDA Graphs. One can see that GPU (orange) is fully utilized.
</figcaption>
</figure>

## [Improvement 4] Running generation in FP8 of the model trained in higher precision 

Implementing FP8 generation with the Gemma model is not straightforward, because this model was initially trained using BF16 precision, and the necessary FP8 scaling factors are missing. Running the model at this lower precision without proper scaling could lead to significant errors and incorrect results.

It is highly recommended to familiarize oneself with the [tutorial](../../examples/fp8_primer.ipynb) on FP8 precision to understand the necessity of scaling.


<figure align="center">
<img src="./media/calibration_1_half.svg">
<figcaption>
    Figure 8: The FP8 scaling factors are incorrect and that leads to numerical errors. The weight calibration allows us to compute FP8 metadata during the forwards in higher precision.
</figcaption>
</figure>

### Weight Calibration

To address the issue outlined above, weight calibration will be used. This involves running several forward iterations at BF16 precision within the context `te.fp8_autocast(enabled=False, calibration=True)`. This setup allows the forward pass to operate at higher precision, while simultaneously collecting `amax_history` and other parameters related to the FP8 precision, which are essential for calculating the FP8 scaling well.

The code below outlines the steps to initialize the BF16 model and conduct several forward iterations within the specified context. After these iterations, the model is saved, and these weights will be utilized in subsequent chapters.

In [1]:
# Restart the notebook (to flush the GPU memory)
from utils import restart_jupyter_notebook

restart_jupyter_notebook()

import transformer_engine.pytorch as te
from utils import *

hyperparams.model_name = "/tmp/gemma-7b-hf/"  # <== Add model weight location here e.g. "/path/to/downloaded/gemma/weights"
hyperparams.fuse_qkv_params = True  # This is needed by the last improvement.

model = init_te_gemma_model(hyperparams)

# Calibration
with te.fp8_autocast(enabled=False, calibrating=True), torch.autocast(
    device_type="cuda", dtype=torch.bfloat16
):
    model.train()
    run_forward_pass(model, hyperparams, num_iters=512)

# Compute scale_fwd with enabled fp8 autocast
with te.fp8_autocast(enabled=True), torch.autocast(
    device_type="cuda", dtype=torch.bfloat16
):
    run_forward_pass(model, hyperparams, 1)

# Some parameters are in pointing to the same tensors, double save is avoided here.
dict_to_save = {
    k: v
    for k, v in model.state_dict().items()
    if ("_context_phase" not in k and "_generation_phase" not in k)
}
torch.save(
    dict_to_save, "calibrated_weights.pth"
)  # <== Add path to save calibrated weights.

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in TEGemmaForCausalLM is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator, or load the model with the `torch_dtype` argument. Example: `model = AutoModel.from_pretrained("openai/whisper-tiny", attn_implementation="flash_attention_2", torch_dtype=torch.float16)`
Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in GemmaModel is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator, or load the model with the

### Generation in FP8

<figure align="center">
<img src="./media/calibration_2_half.svg">
<figcaption>
    Figure 8: After the weight calibration FP8 scaling factors are correct and prevent numerical errors.
</figcaption>
</figure>

Now FP8 inference is ready to be run.

In [ ]:
!ls -alh /perfhome/repos/data/gemma-7b-hf/

In [1]:
# Restart the notebook (to flush the GPU memory)
from utils import restart_jupyter_notebook

restart_jupyter_notebook()

from utils import *

hyperparams.model_name = "/ckpt/gemma-7b-hf/"  # <== Add model weight location here e.g. "/path/to/downloaded/gemma/weights"
hyperparams.fuse_qkv_params = True  # This is needed by the last improvement.
hyperparams.batch_size = 64
hyperparams.fp8 = True
# Calibrated fp8 weights are loaded directly from the file.

hyperparams.fp8_model_weights_filename = (
    "calibrated_weights.pth"  # <== Add calibrated weights location here.
)

hyperparams.generation_cuda_graphs = True
hyperparams.cuda_graphs_static_batch_size = 64
hyperparams.cuda_graphs_static_max_seq_len = 512
hyperparams.cuda_graphs_static_max_context_len = 512
model = init_te_gemma_model(hyperparams)

print_sample_of_generated_texts(model, hyperparams)
benchmark_generation(model, hyperparams)

/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


batch_size: 64,                     max_input_sequence_len: 512,                     lengths: tensor([ 9,  6, 10,  6,  9,  6, 10,  6,  9,  6, 10,  6,  9,  6, 10,  6,  9,  6,
        10,  6,  9,  6, 10,  6,  9,  6, 10,  6,  9,  6, 10,  6,  9,  6, 10,  6,
         9,  6, 10,  6,  9,  6, 10,  6,  9,  6, 10,  6,  9,  6, 10,  6,  9,  6,
        10,  6,  9,  6, 10,  6,  9,  6, 10,  6], device='cuda:0')
============================== Generation example 1 ==============================
Prompt:
Here are the two facts about GPUs:
Generated text:


1. GPUs are very good at doing the same thing to a lot of data.
2. GPUs are very bad at doing different things to a lot of data.

The first fact is why GPUs are so good at graphics. The second fact
============================== Generation example 2 ==============================
Prompt:
Some facts about NVIDIA:
Generated text:


* NVIDIA is a global technology company that designs and builds advanced computer graphics and computing products.
* NVIDIA 

One can observe that the outputs are coherent; however, the generation time has increased. Why is this the case?


<figure align="center">
<img src="./media/fp8_model_init_1_half.svg">
<figcaption>
    Figure 9: Running the model at higher precision involves only one GEMM operation. However, when the model operates in FP8, it requires not just the low-precision GEMM but also weight casting.
</figcaption>
</figure>

Running the model in FP8 does not imply that all weights are stored in FP8. By default, they are stored in higher precision and are cast to FP8, using saved scaling factors, before operations such as GEMMs.

This approach is beneficial during training: one can perform one cast for both backward and forward passes, leading to speedups. However, performing a single cast for each forward pass introduces too much overhead to achieve a speedup. This issue will be addressed in the next section of the tutorial.

### Use of only FP8 model weights

TransformerEngine stores parameters in higher precision and only casts them to FP8. It may be necessary to maintain accucacy during training. However, high precision is not needed when doing inference. 

Transformer Engine supports maintaining only FP8 weights with `fp8_model_init` decorator. Let's see an example
```
linear = te.Linear(1024, 1024) # this module is initialized with full precision weights
with te.fp8_model_init(enabled=True):
    linear_fp8 = te.Linear(1024, 1024) # this module is initialized only with fp8 weights

assert type(linear.weight.data) is torch.Tensor
assert type(linear_fp8.weight.data) is te.float8_tensor.Float8Tensor
```

<figure align="center">
<img src="./media/fp8_model_init_2_half.svg">
<figcaption>
    Figure 10: Using fp8_model_init stores the weights directly in FP8 format, which reduces both time and memory usage.
</figcaption>
</figure>

Let's run the code with `fp8_model_init`:

In [1]:
# Restart the notebook (to flush the GPU memory)
from utils import restart_jupyter_notebook

restart_jupyter_notebook()

# Import necessary packages and methods
from utils import *

hyperparams.model_name = "/ckpt/gemma-7b-hf/"  # <== Add model weight location here e.g. "/path/to/downloaded/gemma/weights"
hyperparams.fuse_qkv_params = True  # Needed for fp8_model_init().
hyperparams.batch_size = 64

hyperparams.fp8 = True
hyperparams.fp8_model_init = True  # This will result in storing only fp8 weights.
hyperparams.fp8_model_weights_filename = (
    "calibrated_weights.pth"  # <== Add calibrated weights location here.
)

hyperparams.generation_cuda_graphs = True
hyperparams.cuda_graphs_static_batch_size = 64
hyperparams.cuda_graphs_static_max_seq_len = 512
hyperparams.cuda_graphs_static_max_context_len = 512
model = init_te_gemma_model(hyperparams)

print_sample_of_generated_texts(model, hyperparams)
benchmark_generation(model, hyperparams)

/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


batch_size: 64,                     max_input_sequence_len: 512,                     lengths: tensor([ 9,  6, 10,  6,  9,  6, 10,  6,  9,  6, 10,  6,  9,  6, 10,  6,  9,  6,
        10,  6,  9,  6, 10,  6,  9,  6, 10,  6,  9,  6, 10,  6,  9,  6, 10,  6,
         9,  6, 10,  6,  9,  6, 10,  6,  9,  6, 10,  6,  9,  6, 10,  6,  9,  6,
        10,  6,  9,  6, 10,  6,  9,  6, 10,  6], device='cuda:0')
============================== Generation example 1 ==============================
Prompt:
Here are the two facts about GPUs:
Generated text:


1. GPUs are very good at doing the same thing to a lot of data.
2. GPUs are very bad at doing different things to a lot of data.

The first fact is why GPUs are so good at graphics. The second fact
============================== Generation example 2 ==============================
Prompt:
Some facts about NVIDIA:
Generated text:


* NVIDIA is a global technology company that designs and builds advanced computer graphics and computing products.
* NVIDIA 

| Models                                                      | Time (s) | Speedup |  
|-------------------------------------------------------------|---------------------------------------|--------------------------------------|
| HF (baseline)                                               | 45.11      | 1                         |
| TE (subsitution of GemmaDecoderLayer with te.TransformerLayer)                                              | 12.10      | 3.7                         | 
| TE (te.TransformerLayer) + CUDA Graphs                                             | 6.41      | 7                         |  
| TE (te.TransformerLayer) + THD attention + FP8                                             | 5.11      | 8.8                         |  

The final speedup is **8.8x**.

## Conclusions


<figure align="center">
<img src="./media/plot.svg">
<figcaption>
    Figure 11: Times obtained with optimizations using TransformerEngine (seconds) along with respective speedup numbers.
</figcaption>
</figure>

In this tutorial, we've explored three features of the Transformer Engine:
1. Support for the THD attention layout,
2. Integration with CUDA Graphs,
3. FP8 weights calibration,
4. Models containing only FP8 version of their parameters.

Each of these features can be applied in various contexts, such as fast token generation. It's important to note that the fastest possible inference speeds can be achieved using NVIDIA's inference-optimized [TensorRT](https://developer.nvidia.com/tensorrt) library.